# Introdução

Nesta atividade, está sendo utilizado o dataset `Credit Card Fraud Detection` para desenvolver um modelo com Keras, visando identificar fraudes em transações de cartões de crédito. O dataset possui 284,807 amostras e 30 features numéricas, resultantes de uma transformação PCA para anonimização. A tarefa é classificar as transações como fraudulentas (1) ou legítimas (0).

# 1) Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso. Envolvendo a instalação de bibliotecas e configuração de outros ajustes necessários. O objetivo é criar um ambiente funcional para executar tarefas específicas.

## 1.1) Instalação das bibliotecas

In [17]:
import pandas as pd

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Accuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score

## 2.2) Leitura dos Dados

In [18]:
BASE_CARTAO = pd.read_csv('/content/creditcard.csv')

In [19]:
BASE_CARTAO

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124856,77448,-0.835868,0.673600,1.904021,-0.263201,0.351435,0.236937,0.223675,0.334808,-0.232802,...,0.114441,0.182323,-0.105240,-0.357454,-0.357218,-0.704835,0.014996,0.144135,4.99,0.0
124857,77448,-2.817937,-0.107162,0.402607,-0.980962,0.682698,0.980801,0.117625,1.387942,-0.779965,...,0.280573,0.323453,-0.374319,-0.962394,0.422271,0.402743,-0.288024,-0.339470,115.00,0.0
124858,77449,-1.262504,1.434804,0.290138,-0.436888,0.801078,0.380510,0.399066,0.554237,-0.531654,...,0.195197,0.383922,-0.281029,-1.152876,0.030816,-0.486364,-0.148485,0.121988,1.07,0.0
124859,77449,1.087845,0.522904,0.275704,2.474583,0.213114,-0.225345,0.389247,-0.100848,-0.960723,...,0.007535,-0.054694,-0.061887,0.083010,0.527815,0.030754,-0.017737,0.020217,45.95,0.0


## 2.3) Exploração da Base

*   **Número de Amostras**: 284,807 transações.
*   **Número de Features**: 30, que incluem features numéricas resultantes de uma transformação PCA para anonimização dos dados. A feature "Class" é o rótulo alvo, onde 1 indica uma transação fraudulenta e 0 uma transação legítima.
*   **Tarefa de Classificação**: Prever se uma transação de cartão de crédito é fraudulenta (1) ou legítima (0).


In [37]:
BASE_CARTAO = BASE_CARTAO.dropna()

In [24]:
BASE_CARTAO.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


# 2) Modelo em Keras

Um modelo sequencial em Keras é uma forma simples de criar redes neurais. Ele permite construir uma pilha linear de camadas, onde cada camada tem exatamente uma entrada e uma saída.



## 2.1) Preparação dos Dados

Inicialmente é realizada a remoção da coluna "Class" e o armazenamento do restante das features em X. Essas features são os atributos que o modelo utilizará para fazer as previsões. Já em y é armazenada a coluna "Class", que representa o rótulo a ser previsto (fraude ou não).


In [38]:
X = BASE_CARTAO.drop(columns=['Class'])
y = BASE_CARTAO['Class']

**Dividindo os dados em conjunto de treino e teste**

É realizada a divisão de X e y em conjuntos de treino (80%) e teste (20%), garantindo a mesma proporção de fraudes e não-fraudes em ambos.

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Normalizando as features**

É realizada a normalização das features para que tenham média 0 e desvio padrão 1, melhorando a performance do modelo.

In [40]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 2.2) Construindo o modelo

Abaixo é criado um modelo sequencial simples, onde é adicionada uma camada densa com 1 neurônio e ativação sigmoide para prever a probabilidade de fraude.

In [41]:
model = Sequential()
model.add(Dense(1, input_shape=(X_train.shape[1],), activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Compilando o modelo**

Abaixo é definido o otimizador Adam, a função de perda binária (binary_crossentropy), e as métricas de avaliação.

In [42]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', AUC(name='AUC')])

**Treinando o modelo**

O modelo é treinado por 50 épocas com batch size 10, usando 20% do treino para validação.

In [43]:
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

Epoch 1/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - AUC: 0.7983 - accuracy: 0.8859 - loss: 0.3394 - val_AUC: 0.9697 - val_accuracy: 0.9988 - val_loss: 0.0159
Epoch 2/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - AUC: 0.9653 - accuracy: 0.9987 - loss: 0.0112 - val_AUC: 0.9550 - val_accuracy: 0.9989 - val_loss: 0.0067
Epoch 3/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - AUC: 0.9434 - accuracy: 0.9987 - loss: 0.0064 - val_AUC: 0.9551 - val_accuracy: 0.9989 - val_loss: 0.0060
Epoch 4/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - AUC: 0.9499 - accuracy: 0.9988 - loss: 0.0059 - val_AUC: 0.9550 - val_accuracy: 0.9990 - val_loss: 0.0060
Epoch 5/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - AUC: 0.9462 - accuracy: 0.9990 - loss: 0.0045 - val_AUC: 0.9552 - val_accuracy: 0.9991 - val_loss: 0.0062
Epoch 6/50
7991/7991 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - AUC: 0.9327 - accuracy: 0.9991 - loss: 0.0047 - val_AUC: 0.9549 - val_accuracy: 0.9989 - val_loss: 0.0053
Epoch 7/50
7991/7991 ━

## 2.3) Teste do Modelo

**Prevendo os rótulos do conjunto de teste**

Abaixo são geradas as previsões no conjunto de teste.

In [44]:
y_pred = model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype(int)

781/781 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


**Calculando a acurácia e a métrica F1**

Acurácia: Indica a porcentagem de previsões corretas em relação ao total de previsões feitas.

In [45]:
accuracy = accuracy_score(y_test, y_pred_labels)

In [47]:
accuracy

0.9985984302418709

Métrica F1: Considera tanto a precisão quanto o recall, sendo especialmente útil para avaliar modelos em datasets desbalanceados. Uma alta métrica F1 indica que o modelo tem um bom equilíbrio entre falsos positivos e falsos negativos.

In [46]:
f1 = f1_score(y_test, y_pred_labels)

In [48]:
f1

0.5977011494252874

# 3) Interpretação dos Resultados

**Desempenho do Modelo**



*   **Acurácia**: O modelo apresentou uma acurácia de 99.86%. Isso significa que, na maioria dos casos, o modelo foi capaz de classificar corretamente as transações como fraudulentas ou legítimas.

* **Métrica F1**: a métrica F1 é relativamente baixa, com um valor de 0.5977. A métrica F1 é uma medida mais robusta em cenários de classes desbalanceadas, como este, onde a classe de interesse (fraude) é muito menos representada do que a classe de não-fraude. Nesse sentido, o valor baixo de F1 indica que, apesar da alta acurácia, o modelo pode estar falhando em identificar transações fraudulentas (falsos negativos) ou está marcando transações legítimas como fraudulentas (falsos positivos).

* **AUC**: durante o treinamento oscilou ao redor de 0.94-0.95, o que é um bom indicador de que o modelo é capaz de distinguir entre as classes. No entanto, esse valor não se reflete em uma alta métrica F1, sugerindo que o modelo pode estar mais focado em maximizar a acurácia geral, possivelmente à custa de uma detecção precisa da classe minoritária (fraude).



**Possíveis Melhorias**



*   **Ajuste de Hiperparâmetros**: Experimentar diferentes taxas de aprendizado, número de camadas, e unidades por camada para encontrar um balanço melhor entre precisão e recall.

*  **Ajustar a Amostra de Dados**: Aumentar o número de amostras da classe minoritária (fraude) para que o modelo tenha mais exemplos de fraudes para aprender ou reduzir o número de amostras da classe majoritária (não-fraude) para evitar que o modelo seja tendencioso em favor da classe majoritária.

*   **Camadas Adicionais**: Adicionar mais camadas densas pode ajudar a capturar padrões mais complexos no conjunto de dados.

